# Can the rosetta estimator distinguish between high stable and low stable?

Use 1enh and engineered version

In [1]:
from nomelt.thermo_estimation.rosetta import minimize_structures
from Bio.PDB import *
import nglview as nv
import ipywidgets
import Bio.PDB.PDBList
from nomelt.thermo_estimation.estimators import ESMFoldDGEstimator, ESMFoldDGArgs
from nomelt.thermo_estimation.rosetta import RosettaMinimizationParameters
parser = PDBParser()


KeyboardInterrupt



In [ ]:
from Bio.PDB import *
import nglview as nv
import ipywidgets

In [ ]:
import tempfile

In [ ]:
tmpdir = tempfile.mkdtemp(dir='./tmp')

### First try the ESM estimator

In [ ]:
rosetta_params = RosettaMinimizationParameters(update_pdb=False)
estimator_args = ESMFoldDGArgs(rosetta_params=rosetta_params)

In [ ]:
ENH1 = "RPRTAFSSEQLARLKREFNENRYLTERRRQQLSSELGLNEAQIKIWFQNKRAKI"
_2P6J = "MKQWSENVEEKLKEFVKRHQRITQEELHQYAQRLGLNEEAIRQFFEEFEQRK"

In [ ]:
estimator = ESMFoldDGEstimator(sequences= [ENH1, _2P6J], ids = ['1enh', '2p6j'])

In [ ]:
estimator.run()

The engineered variant is higher in energy than the WT.... not good. Maybe the structure is bungo

In [ ]:
_2P6J_struc = parser.get_structure('2P6J',estimator.pdb_files_dict['2p6j'])

In [ ]:
nv.show_biopython(_2P6J_struc, gui=True)

ESM thinks the engineered variant is not a 3hb

### Now do jsut rosetta on PDB structs, get the two structure for rosetta from PDB

In [ ]:
pdblist = Bio.PDB.PDBList()

In [ ]:
pdblist.retrieve_pdb_file('1ENH', pdir=tmpdir, file_format='pdb')

In [ ]:
pdblist.retrieve_pdb_file('2P6J', pdir=tmpdir, file_format='pdb')

#### the engineered variant is a solvent model, let's get only one structure

In [ ]:
_2P6J_struc = parser.get_structure('2P6J',tmpdir+'/pdb2p6j.ent')

In [ ]:
model = next(_2P6J_struc.get_models())

In [ ]:
nv.show_biopython(model, gui=True)

In [ ]:
len([a for a in model.get_atoms()])

In [ ]:
from Bio.PDB.PDBIO import PDBIO

In [ ]:
io=PDBIO()
io.set_structure(model)
io.save(tmpdir+'/2p6j_single.pdb')

### Run rosetta on the pdbs

In [ ]:
minimize_structures([tmpdir+'/pdb1enh.ent', tmpdir+'/2p6j_single.pdb'])

### try consensus engineered one.

These sequences are highly similar, so are a good test of how well this estrimator could distinguish in the mutation regime we are working in.

In [ ]:
rosetta_params = RosettaMinimizationParameters(update_pdb=False)
estimator_args = ESMFoldDGArgs(rosetta_params=rosetta_params)

In [ ]:
concensus = "RRKRTTFTKEQLEELEELFEKNRYPSAEEREELAKKLGLTERQVKVWFQNRRAKEKK"

In [ ]:
enh1 = "RPRTAFSSEQLARLKREFNENRYLTERRRQQLSSELGLNEAQIKIWFQNKRAKI"

In [ ]:
estimator = ESMFoldDGEstimator(sequences = [enh1, concensus], ids=['wt', 'concensus'])

In [ ]:
estimator.run()

It seems like it can distinguish between them. Let's make sure the structures are good.

In [ ]:
parser = PDBParser()

In [ ]:
estimator.pdb_files_dict

In [ ]:
wt_struct = parser.get_structure('wt', estimator.pdb_files_dict['wt'])
conc_struct = parser.get_structure('conc', estimator.pdb_files_dict['concensus'])

In [ ]:
nv.show_biopython(wt_struct, gui=True)

In [ ]:
nv.show_biopython(conc_struct, gui=True)

## Can AlphaFold estimator distinguish the two?

In [1]:
from nomelt.thermo_estimation import mAFminDGEstimator, mAFminDGArgs, AlphaFoldArgs
from nomelt.thermo_estimation.rosetta import RosettaMinimizationParameters

import logging
logger = logging.getLogger('root')
logger.setLevel(logging.DEBUG)
logging.info('test')

[2023-08-10 16:58:10,093] [INFO] [real_accelerator.py:133:get_accelerator] Setting ds_accelerator to cuda (auto detect)


2023-08-10 16:58:20.574372: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-10 16:58:20.936547: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
INFO:root:test


In [2]:
af_args= AlphaFoldArgs(
    data_dir='/mmfs1/gscratch/cheme/usr/evankomp/af_reduced_dbs/',
    db_preset='reduced_dbs',
    use_precomputed_msas=False,
    base_executable_path="/mmfs1/home/evankomp/cheme/repos/alphafold/run_singularity.py",
    model_preset='model_4',
    models_to_relax='none'
)
estimator_args=mAFminDGArgs(
    af_params=af_args,
    wdir='./tmp/af_dg/',
    use_relaxed=False,
    num_replicates=50,
    fix_msas=True
)
estimator = mAFminDGEstimator(args=estimator_args)

In [3]:
sequences = [
    'DKRPRTAFSSEQLARLKREFNENRYLTERRRQQLSSELGLNEAQIKIWFQNKRAKIKK',
    'RRKRTTFTKEQLEELEELFEKNRYPSAEEREELAKKLGLTERQVKVWFQNRRAKEKK'
]
ids = ['enh', "consensus-hd"]

In [4]:
estimator = mAFminDGEstimator(args=estimator_args)

In [5]:
outs = estimator.run(sequences=sequences, ids=ids)

I0810 16:59:09.158709 22466734860096 run_singularity.py:136] Binding /mmfs1/gscratch/cheme/usr/evankomp/repos/nomelt/analysis/tmp/af_dg -> /mnt/fasta_path_0
I0810 16:59:09.158797 22466734860096 run_singularity.py:136] Binding /mmfs1/gscratch/cheme/usr/evankomp/repos/nomelt/analysis/tmp/af_dg -> /mnt/fasta_path_1
I0810 16:59:09.158883 22466734860096 run_singularity.py:136] Binding /mmfs1/gscratch/cheme/usr/evankomp/af_reduced_dbs/uniref90 -> /mnt/uniref90_database_path
I0810 16:59:09.158920 22466734860096 run_singularity.py:136] Binding /mmfs1/gscratch/cheme/usr/evankomp/af_reduced_dbs/mgnify -> /mnt/mgnify_database_path
I0810 16:59:09.158952 22466734860096 run_singularity.py:136] Binding /mmfs1/gscratch/cheme/usr/evankomp/af_reduced_dbs -> /mnt/data_dir
I0810 16:59:09.158984 22466734860096 run_singularity.py:136] Binding /mmfs1/gscratch/cheme/usr/evankomp/af_reduced_dbs/pdb_mmcif -> /mnt/template_mmcif_dir
I0810 16:59:09.159014 22466734860096 run_singularity.py:136] Binding /mmfs1/gscr

/mmfs1/home/evankomp/cheme/repos/alphafold/alphafold.sif
singularity run --nv --bind /mmfs1/gscratch/cheme/usr/evankomp/repos/nomelt/analysis/tmp/af_dg:/mnt/fasta_path_0,/mmfs1/gscratch/cheme/usr/evankomp/repos/nomelt/analysis/tmp/af_dg:/mnt/fasta_path_1,/mmfs1/gscratch/cheme/usr/evankomp/af_reduced_dbs/uniref90:/mnt/uniref90_database_path,/mmfs1/gscratch/cheme/usr/evankomp/af_reduced_dbs/mgnify:/mnt/mgnify_database_path,/mmfs1/gscratch/cheme/usr/evankomp/af_reduced_dbs:/mnt/data_dir,/mmfs1/gscratch/cheme/usr/evankomp/af_reduced_dbs/pdb_mmcif:/mnt/template_mmcif_dir,/mmfs1/gscratch/cheme/usr/evankomp/af_reduced_dbs/pdb_mmcif:/mnt/obsolete_pdbs_path,/mmfs1/gscratch/cheme/usr/evankomp/af_reduced_dbs/pdb70:/mnt/pdb70_database_path,/mmfs1/gscratch/cheme/usr/evankomp/af_reduced_dbs/small_bfd:/mnt/small_bfd_database_path,./tmp/af_dg/:/mnt/output,/tmp:/tmp --env NVIDIA_VISIBLE_DEVICES=all --env TF_FORCE_UNIFIED_MEMORY=1 --env XLA_PYTHON_CLIENT_MEM_FRACTION=4.0 /mmfs1/home/evankomp/cheme/repos

I0810 17:03:03.678318 22730553911104 run_singularity.py:136] Binding /mmfs1/gscratch/cheme/usr/evankomp/repos/nomelt/analysis/tmp/af_dg -> /mnt/fasta_path_0
I0810 17:03:03.678402 22730553911104 run_singularity.py:136] Binding /mmfs1/gscratch/cheme/usr/evankomp/repos/nomelt/analysis/tmp/af_dg -> /mnt/fasta_path_1
I0810 17:03:03.678452 22730553911104 run_singularity.py:136] Binding /mmfs1/gscratch/cheme/usr/evankomp/af_reduced_dbs/uniref90 -> /mnt/uniref90_database_path
I0810 17:03:03.678488 22730553911104 run_singularity.py:136] Binding /mmfs1/gscratch/cheme/usr/evankomp/af_reduced_dbs/mgnify -> /mnt/mgnify_database_path
I0810 17:03:03.678519 22730553911104 run_singularity.py:136] Binding /mmfs1/gscratch/cheme/usr/evankomp/af_reduced_dbs -> /mnt/data_dir
I0810 17:03:03.678550 22730553911104 run_singularity.py:136] Binding /mmfs1/gscratch/cheme/usr/evankomp/af_reduced_dbs/pdb_mmcif -> /mnt/template_mmcif_dir
I0810 17:03:03.678594 22730553911104 run_singularity.py:136] Binding /mmfs1/gscr

/mmfs1/home/evankomp/cheme/repos/alphafold/alphafold.sif
singularity run --nv --bind /mmfs1/gscratch/cheme/usr/evankomp/repos/nomelt/analysis/tmp/af_dg:/mnt/fasta_path_0,/mmfs1/gscratch/cheme/usr/evankomp/repos/nomelt/analysis/tmp/af_dg:/mnt/fasta_path_1,/mmfs1/gscratch/cheme/usr/evankomp/af_reduced_dbs/uniref90:/mnt/uniref90_database_path,/mmfs1/gscratch/cheme/usr/evankomp/af_reduced_dbs/mgnify:/mnt/mgnify_database_path,/mmfs1/gscratch/cheme/usr/evankomp/af_reduced_dbs:/mnt/data_dir,/mmfs1/gscratch/cheme/usr/evankomp/af_reduced_dbs/pdb_mmcif:/mnt/template_mmcif_dir,/mmfs1/gscratch/cheme/usr/evankomp/af_reduced_dbs/pdb_mmcif:/mnt/obsolete_pdbs_path,/mmfs1/gscratch/cheme/usr/evankomp/af_reduced_dbs/pdb70:/mnt/pdb70_database_path,/mmfs1/gscratch/cheme/usr/evankomp/af_reduced_dbs/small_bfd:/mnt/small_bfd_database_path,./tmp/af_dg/:/mnt/output,/tmp:/tmp --env NVIDIA_VISIBLE_DEVICES=all --env TF_FORCE_UNIFIED_MEMORY=1 --env XLA_PYTHON_CLIENT_MEM_FRACTION=4.0 /mmfs1/home/evankomp/cheme/repos

INFO:pyrosetta.rosetta:Found rosetta database at: /mmfs1/gscratch/cheme/usr/evankomp/mamba/mambaforge/envs/nomelt2/lib/python3.9/site-packages/pyrosetta-2023.27+release.e3ce6ea9faf-py3.9-linux-x86_64.egg/pyrosetta/database; using it....


PyRosetta-4 2023 [Rosetta PyRosetta4.Release.python39.ubuntu 2023.27+release.e3ce6ea9faf661ae8fa769511e2a9b8596417e58 2023-07-07T12:00:46] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.


INFO:pyrosetta.rosetta:PyRosetta-4 2023 [Rosetta PyRosetta4.Release.python39.ubuntu 2023.27+release.e3ce6ea9faf661ae8fa769511e2a9b8596417e58 2023-07-07T12:00:46] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
INFO:rosetta:core.init: Checking for fconfig files in pwd and ./rosetta/flags
INFO:rosetta:core.init: Rosetta version: PyRosetta4.Release.python39.ubuntu r353 2023.27+release.e3ce6ea9faf e3ce6ea9faf661ae8fa769511e2a9b8596417e58 http://www.pyrosetta.org 2023-07-07T12:00:46
INFO:rosetta:core.init: command: PyRosetta -ex1 -ex2aro -database /mmfs1/gscratch/cheme/usr/evankomp/mamba/mambaforge/envs/nomelt2/lib/python3.9/site-packages/pyrosetta-2023.27+release.e3ce6ea9faf-py3.9-linux-x86_64.egg/pyrosetta/database
INFO:rosetta:basic.random.init_random_generator: 'RNG device' seed mode, using '/dev/urandom', seed=-351868217 seed_offset=0 real_seed=-351868217
INFO:rosetta:basic.random.init_rando

PyRosetta-4 2023 [Rosetta PyRosetta4.Release.python39.ubuntu 2023.27+release.e3ce6ea9faf661ae8fa769511e2a9b8596417e58 2023-07-07T12:00:46] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.


INFO:pyrosetta.rosetta:PyRosetta-4 2023 [Rosetta PyRosetta4.Release.python39.ubuntu 2023.27+release.e3ce6ea9faf661ae8fa769511e2a9b8596417e58 2023-07-07T12:00:46] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
INFO:rosetta:core.init: Checking for fconfig files in pwd and ./rosetta/flags
INFO:rosetta:core.init: Rosetta version: PyRosetta4.Release.python39.ubuntu r353 2023.27+release.e3ce6ea9faf e3ce6ea9faf661ae8fa769511e2a9b8596417e58 http://www.pyrosetta.org 2023-07-07T12:00:46
INFO:rosetta:core.init: command: PyRosetta -ex1 -ex2aro -database /mmfs1/gscratch/cheme/usr/evankomp/mamba/mambaforge/envs/nomelt2/lib/python3.9/site-packages/pyrosetta-2023.27+release.e3ce6ea9faf-py3.9-linux-x86_64.egg/pyrosetta/database
INFO:rosetta:basic.random.init_random_generator: 'RNG device' seed mode, using '/dev/urandom', seed=-987919770 seed_offset=0 real_seed=-987919770
INFO:rosetta:basic.random.init_rando

In [6]:
outs

{'enh': (-137.2163400038844, 5.359040241829916),
 'consensus-hd': (-150.90195406256407, 0.8423209088568342)}